<a href="https://colab.research.google.com/github/redinbluesky/the-lm-book/blob/main/04_트랜스포머.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 목차
* [Chapter 4_0 개요](#chapter4_0)
* [Chapter 4_1 디코더 블록](#chapter4_1)
* [Chapter 4_2 셀프 어텐션](#chapter4_2)
* [Chapter 4_3 위치별 다층 퍼셉트론](#chapter4_3)
* [Chapter 4_4 로터리 위치 임베딩](#chapter4_4)

## Chapter 4_0 개요 <a class="anchor" id="chapter4_0"></a>
1. 트랜스포머 모델은 NLP 분야를 크개 발전시켰다.
    - 넓은 범위의 의존성을 관리하는 데 있어서 RNN의 한계를 극복하고 입력 시퀸스를 병렬로 처리할 수 있다.

2. 트랜스포머 모델에는 세 가지 주요 유형이 있다.
    - 인코더-디코더 트랜스포머: 번역과 같은 시퀸스-투-시퀸스 작업에 사용된다.
    - 인코더 전용 트랜스포머: BERT와 같은 모델로, 주로 이해 작업에 사용된다.
    - 디코더 전용 트랜스포머: GPT 시리즈와 같은 모델로, 주로 생성 작업에 사용된다.

3. 트랜스포머 구조의 핵심 구성 요소는 다음과 같다.
    - 셀프 어텐션 메커니즘: 입력 시퀸스의 다른 위치에 있는 단어들과 얼마나 관련이 있는지 평가한다.
    - 포지셔널 인코딩: 시퀸스 내에서 단어의 순서를 나타낸다.
    - 피드포워드 신경망: 각 위치에서 독립적으로 적용되는 완전 연결 신경망이다.

4. 트랜스포머는 모든 토큰을 동시에 처리한다.
    - 토큰을 병렬 처리함에도 불구하고, 위치 인코딩을 사용해 순차적인 맥락을 유지한다.

5. 디코더 기반 트랜스포머는 아래의 그림처럼 디코더 블록이라는 층을 수직으로 여러 개 쌓아 구성된다.
    - 디코더를 훈련시키려면 입력 시퀸스와 한 토큰씩 앞으로 이동시킨 출력 시퀸스가 필요하다.

        ![Transformer](image/04-00-transformer.png)

## Chapter 4_1 디코더 블록 <a class="anchor" id="chapter4_1"></a>
1. 디코더 블록은 두 개의 하위 층이 있다.
    - 셀프 어텐션과 위치별 다층 퍼셉트론(MLP)이다.

2. 디코더 블록에서 아래와 같은 작업이 이루어진다.
    - 토큰 임베딩을 처리한다.
    - 셀프 어텐션 층은 1부터 L까지 모든 토크 t에 대해 입력 임베딩 벡터 x_t를 세로운 벡터 g_t로 변환한다.
       - L: 입력 길이
    - 셀프 어텟션 다음에는 위치별 MLP가 각 벡터 g_t를 독립적으로 처리한다.
    - 디코더 블록마다 독립적인 파라미터를 가진 MLP가 있다.
    - MLP 하나가 g_t를 입력으로 받아 z_t를 출력한다.
    - MLP가 출력하는 z_t의 개수는 입력 토큰 x_t의 개수와 같다.
    - 다음 출력 벡터 z_t는 다음 디코더 블록의 입력으로 사용된다.
    - 위의 과정이 모든 디코더 블록에서 반복되며, 출력 벡터의 개수는 입력 토큰 x_t의 개수와 같다.
    
        ![Decoder Block](image/04-01-decoder-block.png)

## Chapter 4_2 셀프 어텐션 <a class="anchor" id="chapter4_2"></a>
1. 5개의 토큰을 가진 샘플 ["We","train","a","transformer","model"]과 최대 입력 길이가 4인 디코더가 있다고 가정
    - 각 토큰은 6차원 임베딩 벡터로 표현된다고 가정
    - 각 디코더 블록에서 셀프 어텐션은 훈련 가능한 세 개의 파라미터 행렬 W<sup>q</sup>, W<sup>k</sup>, W<sup>v</sup>를 사용
        - q: 쿼리(query), k: 키(key), v: 값(value)
        - 파라미터 행렬의 텐서의 크기가 6 * 6 이라고 가정
    - 입력 임베딩 벡터(4 * 6)와 훈련 가능한 파라미터(6 * 6)와 곱해져 쿼리 행렬 Q, 키 행렬 K, 값 행렬 V(각각 4*6)가 생성된다.

2. "train" 단어를 표현하는 두 번째 토큰 x<sub>2</sub>에 대한 출력 g<sub>2</sub>을 계산하기 위해 셀프 어텐션 층은 6단계를 거친다.

3. 첫 번째 단계
    - 아래와 그림과 같이 쿼리 행렬 Q, 키 행렬 K, 값 행렬 V를 계산한다.
    - 임베딩 벡터 x<sub>1</sub>, x<sub>2</sub>, x<sub>3</sub>, x<sub>4</sub>에 각각 W<sup>q</sup>, W<sup>k</sup>, W<sup>v</sup>를 곱한다.
    - 행렬 곱셈의 결과로 쿼리 행렬 Q, 키 행렬 K, 값 행렬 V가 생성된다.
    - 이 행렬은 각각 4행 6열의 크기를 가진다.
    - 각각의 입력 임베딩 K<sub>t</sub>는 쿼리 벡터 Q<sub>t</sub>와 값 벡터 V<sub>t</sub>, 키 벡터 K<sub>t</sub>와 매핑된다.
    
        ![Self-Attention](image/04-02-self-attention-1.png)

4. 두 번째 단계
    - 두 번째 토큰 "train"을 처리하는 것을 가정한다.
    - 쿼리 행렬 Q에서 두 번째 행 Q<sub>2</sub>와 키 행렬을 점곱하여 어텐션 점수를 계산한다.
        - Q<sub>2</sub> • K<sub>1</sub> = 4.90
        - Q<sub>2</sub> • K<sub>2</sub> = 17.15
        - Q<sub>2</sub> • K<sub>3</sub> = 9.80
        - Q<sub>2</sub> • K<sub>4</sub> = 12.25
    - 벡터로 나타내면 아래와 같다.
        - 어텐션 점수<sub>2</sub> = [4.90, 17.15, 9.80, 12.25]<sup>T</sup>
    
5. 세 번째 단계
    - 어텐션 점수를 스케일링한다.
    - 어텐션 점수를 키 벡터의 차원 수의 제곱근(√6 ≈ 2.45)으로 나눈다.
    - 스케일된 어텐션 점수<sub>2</sub> =[4.90/2.45, 17.15/2.45, 9.80/2.45, 12.25/2.45]<sup>T</sup>=[2.00, 7.00, 4.00, 5.00]<sup>T</sup>

6. 네 번째 단계
    - 스케일된 어텐션 점수에 코잘 마스크를 적용한다.
        - 코잘 마스크는 미래 토큰에 대한 어텐션 점수를 음의 무한대로 설정한다.
    - 두 번째 입력 위치에 대한 코잘 마스크는 아래와 같다.
        - 코잘 마스크<sub>2</sub> =[0, 0, -∞, -∞]<sup>T</sup>
    - 스케일 조정된 점수에 코잘 마스크를 더해 마스크드 점수를 얻는다.
        - 마스크드 점수<sub>2</sub> = 스케일 조정됨 점수<sub>2</sub> + 코잘 마스크<sub>2</sub> = [2.00, 7.00, -∞, -∞]<sup>T</sup>

7. 다섯 번째 단계
    - 마스크드 점수에 소프트맥스 함수를 적용해 어텐션 가중치를 계산한다.
    - 어텐션 가중치<sub>2</sub> = softmax([2.00, 7.00, -∞, -∞]) 
    - = [e^(2.00)/ (e^(2.00) + e^(7.00)), e^(7.00)/ (e^(2.00) + e^(7.00)), 0.0, 0.0]<sup>T</sup> 
    - = [0.0025, 0.9975, 0.0, 0.0]<sup>T</sup>
    - 어덴션 점수를 키 차원의 제곱근으로 나누면 차원이 커짐에 따라 점곱 결과가 너무 커지는 것을 방지할 수 있다.
    - 큰 점곱과 소프트맥스를 적용하면 그레이디언트가 매우 작아져 학습이 어려워진다.\
    
8. 여섯 번째 단계
    - 어텐션 가중치와 값 벡터 V<sub>1</sub>, V<sub>2</sub>, V<sub>3</sub>, V<sub>4</sub>를 가중합해 출력 벡터 g<sub>2</sub>를 계산한다.
        - g<sub>2</sub> ≈ 0.0025 * V<sub>1</sub> + 0.9975 * V<sub>2</sub> + 0.0 * V<sub>3</sub> + 0.0 * V<sub>4</sub>
    - 위치 2에 대한 디코더의 출력은 위치 1과 위치 2에 있는 입력에만 의존며, 2의 출력에 좀더 큰 가중치를 부여한다.
    - 미래 위치를 참조하지 못하도록 하는 코잘 마스크의 효과이다.
    - 이 예제는 두 번째 토큰이 자기 자신에게 주의를 기울이지만, 문맥에 따라 다른 토큰에 주의를 기울일 수도있다.

9. 벡터 Q, K, V는 다음과 같이 이해할 수 있다.
    - 쿼리 벡터 Q<sub>t</sub>
        - 토큰 임베딩은 다른 위치에 대한 정보를 찾는다.
        - 예를 들어 "I"라는 단어는 "am"이라는 단어를 찾을 수 있다.
        - 모든 위치 t를 위한 쿼리 벡터 Q<sub>t</sub>는 해당 위치에서 찾고자 하는 정보를 나타낸다.
    - 키 벡터 K<sub>t</sub>
        - Q<sub>t</sub>와 각 위치에 있는 K<sub>t</sub>를 점곱해 두 벡터 사이의 유사도를 계산한다.
        - 점곱 결과가 클수록 두 벡터가 유사하다는 의미이다.
    - 값 벡터 V<sub>t</sub>
        - 어떤 위치 t의 K<sub>t</sub>가 쿼리 벡터 Q<sub>i</sub>와 유사하다면, 해당 위치 t의 값 벡터 V<sub>t</sub>가 출력에 더 큰 영향을 미친다.

10. 어텐션 개념은 트랜스포머가 나오기 전에 등장했다.
    - 2014년 요수아 벤지오의 지도하에서 연구하던 드미트리 바흐나나흐는 번역 작업에서 어텐션 메커니즘을 사용한 최초의 논문을 발표했다.
    - 바흐다나우는 영어를 배울 때 문장의 여러 다른 부분에 초점을 맞췄던 자신의 학습 경험을 토대로 RNN이 각 번역 단계에서 가장 주요한 입력 단어에 집중할 수 있도록 어텐션 메커니즘을 고안했다.

11. g<sub>2</sub> 계산에 사용된 과정이 입력 시퀸스에 있는 다른 위치에도 반복되어 출력 벡터 g<sub>1</sub>, g<sub>3</sub>, g<sub>4</sub>가 생성된다.
    - 이 경우 코잘 마스크는 아래와 같이 적용된다.
        - g<sub>1</sub>: 코잘 마스크<sub>1</sub> =[0, -∞, -∞, -∞]<sup>T</sup>
        - g<sub>3</sub>: 코잘 마스크<sub>3</sub> =[0, 0, 0, -∞]<sup>T</sup>
        - g<sub>4</sub>: 코잘 마스크<sub>4</sub> =[0, 0, 0, 0]<sup>T</sup>
    - 첫 번째 토큰은 자기 자신에게만 주의를 기울이고, 두 번째 토큰은 첫 번째와 두 번째 토큰에만 주의를 기울인다.

12. 모든 위치에 대한 어텐션을 계산하는 일반화된 공식은 아래와 같다.
    - 어텐션(Q, K, V) = softmax( (Q • K<sup>T</sup>) / √d<sub>k</sub> + M ) • V
        - d<sub>k</sub>: 키 벡터의 차원 수
        - M: 코잘 마스크 행렬
        - Q, V: (L, d<sub>model</sub>) 크기의 행렬
        - K: (L, d<sub>k</sub>) 크기의 행렬
        - L: 입력 길이
    - QK<sup>T</sup>는 한번에 모든 어텐션 점수를 계산하므로 훨씬 빠르다.

## Chapter 4_3 위치별 다층 퍼셉트론 <a class="anchor" id="chapter4_3"></a>
1. 마스크드 셀프 어텐션 층 다음에 출력 벡터 g<sub>t</sub>가 다층 퍼셉트론(MLP)에서 독립적으로 처리된다.
    - 일부 자료에서는 피드포워드 신경망, 완전 연결층 이라고 부른다.
    - 아래와 같 식의 변환을 적용한다.
        - z<sub>t</sub> = W<sub>2</sub>(Relu(W<sub>1</sub>g<sub>t</sub> + b<sub>1</sub>) + b<sub>2</sub>)
            - W<sub>1</sub>, W<sub>2</sub>: 훈련 가능한 가중치 행렬
            - b<sub>1</sub>, b<sub>2</sub>: 훈련 가능한 바이어스 벡터
    - Z<sub>t</sub>는 다음 디코더 블록의 입력으로 사용된다.

## Chapter 4_4 로터리 위치 임베딩 <a class="anchor" id="chapter4_4"></a>
1. 트랜스포머는 순차적인 의존성 없이 모든 토큰에 대해 한 번에 어텐션 점수를 계산한다.
    - 그러나 자연어는 순차적인 특성을 가지므로, 토큰의 순서를 모델에 제공해야 한다.
    - 이를 위해 트랜스포머는 위치 임베딩을 사용한다.

2. 단어의 순서를 고려하려면 트랜스포머는 위치 정보를 통합해야 한다.

3. 널리 사용되는 방법은 로터리 위치 임베딩(RoPE, Rotary Position Embedding)이다.
    - 어텐션 메커니즘 안에서 쿼리 벡터와 키 벡터를 위치에 따라 회전시킨다.
    - 훈련 과정에서 본 것보다 더 킨 시퀸스를 효과적으로 일반화 할 수 있다.
    - 시간과 자원을 절약하기 위해 짧은 시퀸스에서 훈련 후 추론 할 때 긴 시퀸스를 처리할 수 있다.

4. RoPE는 쿼리 벡터와 키 키 벡터를 회전하여 위치 정보를 인코딩한다.
    - 어텐션을 계산하기 전에 이 회전이 수행된다.

5. 이미지로 표현하면 아래와 같다.
    - "원본"이라고 표시된 검은 화살표는 셀프 어텐션에 있는 위치 정보가 없는 키 벡터나 쿼리 벡터를 나타낸다.
    - RoPE는 이 벡터를 토큰 위치에 따라 회전 시킴으로써 위치 정보를 임베딩한다.
        - 쿼리와 키 벡터 전체를 회전시키는 것이 아니라 벡터 안에 있는 인접한 차원 쌍을 회전시킨다.
    - 색깔이 있는 화살표는 위치 1, 3, 5, 7에 대해 회전된 결과 벡터를 나타낸다.
    - RoPE의 핵심 특징은 회전한 두 벡터 사이의 각도가 시퀸스에 있는 두 벡터 사이의 거리를 인코딩한 것이다.
        - 위치 1과 위치 3 사이의 각도는 위치 5와 위치 7 사이의 각도와 같다.
        - 위치 사이의 거리가 동일하게 두 위치만큼 떨어져 있기 때문이다.

        ![RoPE](image/04-03-rope.png)

6. 회전을 위해 회전 행렬을 사용한다.
    - 회전 행렬은 벡터의 각도와 크기를 유지하면서 벡터를 회전시킬 때 사용하는 수학적 도구이다.
    - RoPE는 쿼리 벡터와 키 벡터의 인접한 두 차원에 회전 행렬을 적용한다.
        - 원점을 중심으로 θ만큼 회전시킨다.
    - 각 차원 쌍에 대해 서로 다른 각도를 사용하여 위치 정보를 인코딩한다.
        - (x, y)를 (x', y')로 변환한다.        
    - 위의 그림에서 원본 벡터의 위치가 4라고 가정한다.
        - 위치 1, 3과 4의 거리와 위치 5, 7과 4의 거리가 같다.
        - 회전 행렬을 사용하는 경우 위치 1,3 사이의 각도와 위치 5,7 사이의 각도가 같다.

7. 2차원 백터 q = [2, 1]<sup>T</sup>를 회전한다고 가정한다.
    - 2차원의 경우 각도 θ에 대한 회전 행렬 R(θ)는 아래와 같다.
        - R(θ) = [[cosθ, -sinθ], [sinθ, cosθ]]
    - 이 벡터를 회전시키려면  q와 회전 행렬 R(θ)를 곱한다.
    - 만들어진 새로운 벡터는 각도 θ만큼 반시계 방향으로 회전한 q'를 나타낸다.
    - 45도 회전(θ = π/4 라디안)의 경우 특별히 아래의 식을 사용할 수 있다
        - cos(π/4) = sin(π/4) = √2/2 ≈ 0.7071
    - 45도의 회전 행렬은 아래와 같다.
        - R(π/4) = [[√2/2, -√2/2], [√2/2, √2/2]]
    - 따라서 벡터 q를 45도 회전시키면 아래와 같다.
    - q' = R(π/4) • q = [[√2/2, -√2/2], [√2/2, √2/2]] • [2, 1]<sup>T</sup>
    - = [(2√2/2) - (1√2/2), (2√2/2) + (1√2/2)]<sup>T</sup> = [0.7071, 2.1213]<sup>T</sup>
    - 그림의로 표현하면 아래와 같다.

        ![Rotation Example](image/04-04-rotation-example.png)

8. 위치 t에서 RoPE는 다움과 같은 쿼리와 키 벡터 안에 있는 차원 쌍을 회전시킨다.
    - q<sub>t</sub> = [q<sub>t</sub><sup>(1)</sup>, q<sub>t</sub><sup>(2)</sup>, q<sub>t</sub><sup>(3)</sup>, q<sub>t</sub><sup>(4)</sup>, ... , q<sub>t</sub><sup>(d<sub>q</sub>-1)</sup>, q<sub>t</sub><sup>(d<sub>q</sub>)</sup>]<sup>T</sup>
    - k<sub>t</sub> = [k<sub>t</sub><sup>(1)</sup>, k<sub>t</sub><sup>(2)</sup>, k<sub>t</sub><sup>(3)</sup>, k<sub>t</sub><sup>(4)</sup>, ... , k<sub>t</sub><sup>(d<sub>k</sub>-1)</sup>, k<sub>t</sub><sup>(d<sub>k</sub>)</sup>]<sup>T</sup>
        - 여기서 d<sub>q</sub>와 d<sub>k</sub>는 각각 쿼리 벡터와 키 벡터의 차원으로 크기가 같다.
    - RoPE는 (2p-1, 2p) 차원 쌍을 회전시킨다.
        - 인덱스 p의 범위는 1 ≤ p ≤ d<sub>q</sub>/2 또는 1 ≤ p ≤ d<sub>k</sub>/2 이다.
    - q<sub>t</sub> 차원을 d<sub>q</sub> / 2개의 쌍으로 나눈다.
        - [q<sub>t</sub><sup>(1)</sup>, q<sub>t</sub><sup>(2)</sup>]<sup>T</sup>, [q<sub>t</sub><sup>(3)</sup>, q<sub>t</sub><sup>(4)</sup>]<sup>T</sup>, ... , [q<sub>t</sub><sup>(d<sub>q</sub>-1)</sup>, q<sub>t</sub><sup>(d<sub>q</sub>)</sup>]<sup>T</sup>
    - q(p)는 [q<sub>t</sub><sup>(2p-1)</sup>,q<sub>t</sub><sup>(2p)</sup>]<sup>T</sup> 쌍을 나타낸다.
    - q<sub>t</sub>(3)는 다음과 같다.
        - [q<sub>t</sub><sup>(2•3-1)</sup>, q<sub>t</sub><sup>(2•3)</sup>] = [q<sub>t</sub><sup>(5)</sup>, q<sub>t</sub><sup>(6)</sup>]<sup>T</sup>

9. 각 쌍 p는 다음 토큰 위치 t와 회전 프리퀸시 θ<sub>p</sub>를 기반으로 회전한다.
    - RoPE(q<sub>t</sub>(p), t) = [cos(θ<sub>p</sub>•t)  -sin(θ<sub>p</sub>•t); sin(θ<sub>p</sub>•t)  cos(θ<sub>p</sub>•t)] • [q<sub>t</sub><sup>(2p-1)</sup>, q<sub>t</sub><sup>(2p)</sup>]<sup>T</sup>
    - 행렬-벡터 곱셈 규칙을 적용하면 회전결과는 다음과 같은 차원 백터가 된다.
        - RoPE(q<sub>t</sub>(p), t) = [q<sub>t</sub><sup>(2p-1)</sup>•cos(θ<sub>p</sub>•t) - q<sub>t</sub><sup>(2p)</sup>•sin(θ<sub>p</sub>•t), q<sub>t</sub><sup>(2p-1)</sup>•sin(θ<sub>p</sub>•t) + q<sub>t</sub><sup>(2p)</sup>•cos(θ<sub>p</sub>•t)]<sup>T</sup>
        - 여기서 θ<sub>p</sub>는 p번재 쌍에 대한 프리퀴시 이며 다음과 같이 정리된다.
            - θ<sub>p</sub> = 1 / ⊖^(2(p-1)/d<sub>q</sub>)
                - ⊖: 사전 정의된 상수로 초기에는 10,000으로 설정되었으나 나중에 실험을 통해 값이 더 커지면 문캑 크기를 더 늘릴 수 있다는 것이 밝혀졌다.
    
10. 회전된 전체 임베딩 RoPE(q<sub>t</sub>)는 회전된 쌍을 모두 연결해 구성된다.
    - RoPE(q<sub>t</sub>) = concate[RoPE(q<sub>t</sub>(1)), RoPE(q<sub>t</sub>(2)), ... , RoPE(q<sub>t</sub>(d<sub>q</sub>/2))]
    - 분모에 지수 항이 있으므로 회전 프리퀸시 θ<sub>p</sub>는 p가 커짐에 따라 점점 작아진다.

11. RoPE는 회전이 많이 되는 초반부 차원에서는 세밀하게 국부적인 위치 정보를 포착하고, 회전이 느리져는 후반부 차원에서는 듬성듬성 화게 전영 위정보를 포착한다.
    - 이런 조합이 위치 임베딩 표현을 풍부하게 하며, 모든 차원에 하나의 회전 프리퀸시를 사용하는 것보다 모델이 시퀸스에 있는 토큰 위치를 더 효과적으로 구분할 수 있게 만든다.

12. 위치 t에서 6아춴 쿼리 벡터와 ⊖ = 10,000를 가정한다.
    - q<sub>t</sub> = [q<sub>t</sub><sup>(1)</sup>, q<sub>t</sub><sup>(2)</sup>, q<sub>t</sub><sup>(3)</sup>, q<sub>t</sub><sup>(4)</sup>, q<sub>t</sub><sup>(5)</sup>, q<sub>t</sub><sup>(6)</sup>]<sup>T</sup> = [0.8, 0.6, 0.7, 0.3, 0.5, 0.4]<sup>T</sup>
    - 이 벡터를 세 새의 쌍(d<sub>q</sub> / 2 = 3)으로 나눈다.
        - q<sub>t</sub>(1) = [q<sub>t</sub><sup>(1)</sup>, q<sub>t</sub><sup>(2)</sup>]=[0.8, 0.6]<sup>T</sup>
        - q<sub>t</sub>(2) = [q<sub>t</sub><sup>(3)</sup>, q<sub>t</sub><sup>(4)</sup>]=[0.7, 0.3]<sup>T</sup>
        - q<sub>t</sub>(3) = [q<sub>t</sub><sup>(5)</sup>, q<sub>t</sub><sup>(6)</sup>]=[0.5, 0.4]<sup>T</sup>
    - 각 쌍 q는 θ<sub>p</sub>t각도만큼 회전한다.
        - θ<sub>p</sub> = 1 / ⊖^(2(p-1)/d<sub>q</sub>) = 1 / 10,000^(2(p-1)/6)
    -위치 t를 100으로 가정한다.
        - θ<sub>1</sub> = 1 / 10000<sup>2(1-1)/6</sup> = 1 / 10000<sup>0</sup> = 1.0
        - θ<sub>2</sub> = 1 / 10000<sup>2(2-1)/6</sup> = 1 / 10000<sup>1/3</sup> ≈ 0.0464
        - θ<sub>3</sub> = 1 / 10000<sup>2(3-1)/6</sup> = 1 / 10000<sup>2/3</sup> ≈ 0.00215
    - 각 쌍을 회전시킨다.
        - RoPE(q<sub>100</sub>(1)) = [cos(100), -sin(100)] 
                                     [sin(100), cos(100) ] • [0.8, 0.6]<sup>T</sup> ≈ [0.86  0.51
                                                                                      -0.51 0.86] • [0.8, 0.6]<sup>T</sup> = [0.99, 0.113]<sup>T</sup>
        - RoPE(q<sub>100</sub>(2)) = [cos(4.64), -sin(4.64)] 
                                     [sin(4.64), cos(4.64) ] • [0.7, 0.3]<sup>T</sup> ≈ [-0.06  0.998
                                                                                      -0.998 -0.06] • [0.7, 0.3]<sup>T</sup> = [0.233, -0.713]<sup>T</sup>
        - RoPE(q<sub>100</sub>(3)) = [cos(0.215), -sin(0.215)] 
                                     [sin(0.215), cos(0.215) ] • [0.5, 0.4]<sup>T</sup> ≈ [0.98  -0.213
                                                                                           0.213 0.98 ] • [0.5, 0.4]<sup>T</sup> = [0.412, 0.05]<sup>T</sup>          
    - 그래프로 나타내면 아래와 같다.

        ![RoPE Example](image/04-05-rope-example.png)

13. 마지막으로 회전된 쌍을 모두 연결해 최종 회전 임베딩 RoPE(q<sub>100</sub>)를 얻는다.
    - RoPE(q<sub>100</sub>) = concate[RoPE(q<sub>100</sub>(1)), RoPE(q<sub>100</sub>(2)), RoPE(q<sub>100</sub>(3))]
    - = [0.99, 0.113, 0.233, -0.713, 0.412, 0.05]<sup>T</sup>

14. RoPRE<sub>k</sub>도 동일한 방식으로 계산된다.
    - 디코더 블록의 셀프 어텐션 메커니즘에서 쿼리(Q)와 키(K) 행렬에 RoPE를 적용한다.

15. 값 벡터(V)는 어텐션 가중치가 결정된 후에 쿼리와 키가 결합되어 선택한 정보만 제공한다.
    - 위치 관계는 이미 쿼리, 키 벡터에서 포작되었으므로 값 벡터는 로터리 임베딩이 필요하지 않다.

16. RoPE가 적용된 이미지는 아래와 같다.
    - 디코더 블록의 입력과 가중치 행렬 W<sup>Q</sup>, W<sup>K</sup>를 곱하여 Q와 K를 만든 후 어텐션 가중이를 계산하기 전에 RoPE를 적용한다.
    - RoPE는 모든 디코더 블록에 적용되어 위치 정보가 네트워크를 통해 일관성있게 전달되도록 한다.
    - 두 번째 디코드 블록의 출력을 사용해 각 위치에 대한 로짓을 계산한다.
    - 이 값은 마지막 디코더 블록의 출려과(임베딩 차원, 어휘사전 크기)의 행렬을 곱하여 구한다.
    - 이 행렬은 모든 위치에서 공유된다.

    ![RoPE in Self-Attention](image/04-06-rope-in-self-attention.png)